## Import Libraries

In [2]:
import pandas as pd
import numpy as np
import sqlite3

pd.options.display.max_rows = 999
pd.options.display.max_columns = 100

## Import Data

In [3]:
con = sqlite3.connect("db/votes.db")

dfmf = pd.read_sql_query("SELECT * FROM votes", con)
dfmf.head()

,voter_id,county,precinct,candidate
0,0,Palo Alto,WB,Buttigieg
1,1,Lucas,POM,Buttigieg
2,2,Bremer,TR,Warren
3,3,Cerro Gordo,CL,Warren
4,4,Des Moines,DM,Klobuchar


In [5]:
dfmf.candidate.unique()

array(['Buttigieg', 'Warren', 'Klobuchar', 'Biden', 'Yang', 'Sanders',
       'Steyer', 'Uncommitted', 'Other', 'Gabbard'], dtype=object)

In [6]:
cd_map = pd.read_csv('data/Delegate Apportionment_Page 1_Pivot table.csv')
cd_map.columns = ['county','precinct_short','delegates_cc','sde']
cd_map.rename(columns={'precinct_short':'precinct'}, inplace=True)
del cd_map['delegates_cc']
cd_map.head()

,county,precinct,sde
0,Lee,FM4B,0.000
1,Lee,KE2B,0.000
2,Lee,HR,0.000
3,Appanoose,CW,0.160
4,Black Hawk,S000,0.404


In [7]:
cd_map.to_csv('data/delegate_apportionment_clean.csv')

In [ ]:
sqlite_insert_query = """INSERT INTO movies
                  (movie_title, release_year, plot_description, genre, average_rating, number_of_votes) 
                  VALUES (?, ?, ?, ?, ?, ?);"""
recordList = df.get_values()
cursor.executemany(sqlite_insert_query, recordList)
conn.commit()

## What are the first results?

In [50]:
dt = dfmf.copy()
dt = dt.merge(cd_map, how='left', on=['county','precinct'])

In [80]:
def compute_candidate_data(dat):
    dat.loc[:,'cand_vote_sum'] = (dat.groupby(['county','precinct','candidate'])['voter_id']
                                     .transform(lambda x: x.nunique()))
    dat.loc[:,'cand_vote_ttl'] = dat.groupby(['county','precinct'])['voter_id'].transform(lambda x: x.nunique())
    dat.loc[:,'cand_vote_share'] = dat['cand_vote_sum'] / dat['cand_vote_ttl']
    dat.loc[:,'viable_candidate'] = np.where(dat['cand_vote_share'] >= 0.15, 1, 0)
    return dat

In [81]:
dt = compute_candidate_data(dt)

## Who has to re-vote?

In [83]:
dt.loc[dt['viable_candidate']==0,:].head()

,voter_id,county,precinct,candidate,delegates_cc,sde,cand_vote_sum,cand_vote_ttl,cand_vote_share,viable_candidate
4,4,Des Moines,DM,Klobuchar,3,1.080000,16,110,0.145455,0
5,5,Dallas,WM225,Biden,8,3.314286,12,115,0.104348,0
6,6,Van Buren,KE,Yang,10,1.000000,4,125,0.032000,0
8,8,Story,4,Warren,8,3.320000,16,115,0.139130,0
12,12,Scott,MC,Biden,1,0.413333,10,106,0.094340,0


## Pretend that the people re-voted for only viable?

In [127]:
fnl = dt.copy()
fnl.loc[:,'orig_candidate'] = fnl.loc[:,'candidate']
fnl.loc[:,'candidate'] = np.where(fnl['viable_candidate']==1, fnl['candidate'], None)

In [128]:
vc = (fnl.loc[(fnl['viable_candidate']==1),['county','precinct','candidate']]
         .drop_duplicates()
         .groupby(['county','precinct'])['candidate']
         .apply(lambda x: list(x))
         .reset_index())
vc.rename(columns={'candidate':'viable_candidate_list'}, inplace=True)

In [129]:
fnl_agg = fnl.merge(vc, how='left', on=['county','precinct'])
fnl_agg.loc[:,'potential_candidate'] = (fnl_agg['viable_candidate_list'].map(
                                            lambda s: np.random.choice(
                                                str(s).translate(str.maketrans("","","[]' ")).split(','))))
fnl_agg.loc[:,'candidate'] = np.where(fnl_agg['viable_candidate']==1,
                                      fnl_agg['candidate'],
                                      fnl_agg['potential_candidate'])

In [130]:
fnl_agg = compute_candidate_data(fnl_agg)
fnl_agg.head()

,voter_id,county,precinct,candidate,delegates_cc,sde,cand_vote_sum,cand_vote_ttl,cand_vote_share,viable_candidate,orig_candidate,viable_candidate_list,potential_candidate
0,0,Palo Alto,WB,Buttigieg,8,0.666667,44,106,0.415094,1,Buttigieg,"[Buttigieg, Sanders, Warren]",Buttigieg
1,1,Lucas,POM,Buttigieg,5,0.363636,47,150,0.313333,1,Buttigieg,"[Buttigieg, Warren, Sanders, Biden]",Sanders
2,2,Bremer,TR,Warren,5,1.133333,36,119,0.302521,1,Warren,"[Warren, Sanders, Buttigieg]",Warren
3,3,Cerro Gordo,CL,Warren,7,1.240000,43,134,0.320896,1,Warren,"[Warren, Buttigieg, Sanders]",Sanders
4,4,Des Moines,DM,Buttigieg,3,1.080000,56,110,0.509091,1,Klobuchar,"[Sanders, Buttigieg]",Buttigieg


In [151]:
res = fnl_agg[['county','precinct','delegates_cc','sde','candidate','cand_vote_share']].drop_duplicates()
res.loc[:,'cand_sde'] = res['sde'] * res['cand_vote_share']
res = res.groupby('candidate')['cand_sde'].sum().reset_index()
res.loc[:,'cand_share_sde'] = res['cand_sde'] / res['cand_sde'].sum()

In [152]:
res.head()

,candidate,cand_sde,cand_share_sde
0,Biden,168.013259,0.079741
1,Buttigieg,653.028006,0.309933
2,Klobuchar,177.291319,0.084144
3,Sanders,723.953440,0.343594
4,Warren,384.713976,0.182589
